<a href="https://colab.research.google.com/github/crew-guy/ml-with-tensorflow/blob/main/transfer_learning_tf_feature_extraction_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning with Tensorflow Part 1: Feature Extraction

Transfer learning is the process of leveraging an existing model architecture & patterns it has learnt, for our own problem

There are 2 main benefits
1. Can use existing neural network architecture that is proven to work on a problem, similar to our own
2. Can leverage a neural network architecture & patterns which it has already learnt on similar data to our own, then we can adapt those patterns to our own data


In [2]:
# Are we running a GPU
!nvidia-smi

Tue Nov 29 11:44:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading & becoming one with the data

We will only be working with 10% of the dataset for training

In [5]:
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip


# Unzip the data
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2022-11-29 11:46:29--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  41.9MB/s    in 4.5s    

2022-11-29 11:46:34 (35.5 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [9]:
import os

for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
    print(f"There are {len(dirnames)} directories and {len(filenames)} files in dirpath:{dirpath} ")

There are 2 and 0 in dirpath:10_food_classes_10_percent 
There are 10 and 0 in dirpath:10_food_classes_10_percent/train 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/fried_rice 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/sushi 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/steak 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/hamburger 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/grilled_salmon 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/ramen 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/pizza 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/ice_cream 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/chicken_curry 
There are 0 and 75 in dirpath:10_food_classes_10_percent/train/chicken_wings 
There are 10 and 0 in dirpath:10_food_classes_10_percent/test 
There are 0 and 250 in dirpath:10_food_classes_10_percent/test/fried_rice 
The

## Creating data loaders (preparing the data)

We'll use the `ImageDataGenerator` class to load in images in batches

In [15]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [25]:
# Creating data generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

training_datagen_augmented = ImageDataGenerator(
    rescale=1/255.,
    shear_range=0.2,
    rotation_range=0.2,
    height_shift_range=0.2,
    width_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True
)

In [27]:
# Creating data sets for training and testing

IMAGE_SIZE= (224,224)
BATCH_SIZE = 32

train_data = training_datagen.flow_from_directory(
    directory=train_dir,
    target_size=IMAGE_SIZE,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
)

train_data_augmented = training_datagen_augmented.flow_from_directory(
    directory=train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE, 
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    directory=test_dir,
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SIZE,
    class_mode='categorical'
)

Found 750 images belonging to 10 classes.
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [28]:
train_data[7]

(array([[[[0.8431373 , 0.6392157 , 0.05882353],
          [0.83921576, 0.6117647 , 0.0509804 ],
          [0.7843138 , 0.5372549 , 0.        ],
          ...,
          [0.18431373, 0.10980393, 0.08627451],
          [0.19215688, 0.11764707, 0.09411766],
          [0.20000002, 0.1254902 , 0.10196079]],
 
         [[0.8588236 , 0.59607846, 0.        ],
          [0.8313726 , 0.5764706 , 0.        ],
          [0.81568635, 0.5764706 , 0.02745098],
          ...,
          [0.19607845, 0.12156864, 0.10588236],
          [0.19215688, 0.11764707, 0.10196079],
          [0.20392159, 0.12941177, 0.10588236]],
 
         [[0.8941177 , 0.60784316, 0.00392157],
          [0.8313726 , 0.5764706 , 0.01176471],
          [0.8078432 , 0.5882353 , 0.14509805],
          ...,
          [0.19607845, 0.1254902 , 0.11764707],
          [0.18823531, 0.11764707, 0.10980393],
          [0.18431373, 0.1137255 , 0.09803922]],
 
         ...,
 
         [[0.82745105, 0.69411767, 0.8235295 ],
          [0.82745